In [50]:
# !git clone https://github.com/ultralytics/yolov5  # clone
# %cd yolov5
# !pip install -r requirements.txt  # install

In [51]:
%cd ..

/Users/hare/Notebooks/Championships/CP/EKB


In [341]:
import os
import shutil
from tqdm import tqdm
from random import shuffle
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from glob import glob

import torch

from sklearn.model_selection import train_test_split

import ast

# Подготовка данных

In [342]:
TARGETS = {
    'teeth': 0, 
    'caries': 1, 
}

In [343]:
data = pd.read_csv('data.csv')

In [344]:
# bbox - xmin, ymin, xmax, ymax
# yolo - x_c, y_c, width, height

# center_x = ( (xmax + xmin) / 2) / width
# center_y = ( (ymax + ymin) / 2) / height
# width = (xmax - xmin) / width
# height = (ymax - ymin) / height

In [345]:
data['x_c'] = (data.x_max + data.x_min) / (2 * data.width)
data['y_c'] = (data.y_max + data.y_min) / (2 * data.height)
data['width_box'] = (data.x_max - data.x_min) / data.width
data['height_box'] = (data.y_max - data.y_min) / data.height
data['yolo'] = data.apply(lambda x: '{} '.format(str(TARGETS[x[3]]) + ' ' + ' '.join([str(round(y, 4)) for y in x[-4:]])), axis=1)

In [346]:
data['file'] = data.path.apply(lambda x: x.split('/')[-1])

# Подгрузка ручной разметки

In [347]:
data2 = pd.concat([pd.read_csv('via_region_data.csv'),
                  pd.read_csv('via_region_data-2.csv')], ignore_index=True)
data2 = data2.rename(columns={'#filename': 'path'})

data2.region_shape_attributes = data2.region_shape_attributes.apply(lambda x: ast.literal_eval(x))

data2 = data2[data2.region_shape_attributes.apply(lambda x: len(x)) > 0].reset_index(drop=True)

In [348]:
data2['x_min'] = data2.region_shape_attributes.apply(lambda x: np.min(x['all_points_x']))
data2['x_max'] = data2.region_shape_attributes.apply(lambda x: np.max(x['all_points_x']))
data2['y_min'] = data2.region_shape_attributes.apply(lambda x: np.min(x['all_points_y']))
data2['y_max'] = data2.region_shape_attributes.apply(lambda x: np.max(x['all_points_y']))

In [349]:
data2['file'] = data2.path.values
data2.path = data2.path.apply(lambda x: 'data/images/'+x)
data2['label'] = 'caries'

In [350]:
data2 = pd.merge(data2, data[['path', 'height', 'width']].drop_duplicates(),
                 on='path', how='left')

In [351]:
data2 = data2[['path', 'height', 'width', 'label', 'file', 'x_min', 'x_max', 'y_min', 'y_max']]

In [352]:
data2['x_c'] = (data2.x_max + data2.x_min) / (2 * data2.width)
data2['y_c'] = (data2.y_max + data2.y_min) / (2 * data2.height)
data2['width_box'] = (data2.x_max - data2.x_min) / data2.width
data2['height_box'] = (data2.y_max - data2.y_min) / data2.height
data2['yolo'] = data2.apply(lambda x: '{} '.format(str(TARGETS[x[3]]) + ' ' + ' '.join([str(round(y, 4)) for y in x[-4:]])), axis=1)



In [353]:
data2['pose'] = 'in'

In [354]:
data_all = pd.concat([
    data[~((data.path.isin(data2.path.unique()))&(data.label == 'caries'))], 
    data2], ignore_index=True)

# Подгрузка ручной разметки 2

In [355]:
import json

In [356]:
data3 = pd.DataFrame()

for file in glob('ann/*.json'):

    with open(file) as f:
        json_ann = json.loads(f.read())

    for i in range(len(json_ann['objects'])):
        
        temp = pd.DataFrame()
        
        xs = []
        ys = []
        
        for point in json_ann['objects'][i]['points']['exterior']:
            xs.append(point[0])
            ys.append(point[1])
            
        temp['file'] = [file.split('/')[-1].split('.json')[0]]
        temp['path'] = 'data/images/' + temp['file']
        temp['label'] = 'caries'
        temp['height'] = 256
        temp['width'] = 256
        
        temp['x_min'], temp['x_max'], temp['y_min'], temp['y_max'] = np.min(xs), np.max(xs), np.min(ys), np.max(ys)
        
        temp = temp[['path', 'height', 'width', 'label', 'file', 'x_min', 'x_max', 'y_min', 'y_max']]
        
        temp['x_c'] = (temp.x_max + temp.x_min) / (2 * temp.width)
        temp['y_c'] = (temp.y_max + temp.y_min) / (2 * temp.height)
        temp['width_box'] = (temp.x_max - temp.x_min) / temp.width
        temp['height_box'] = (temp.y_max - temp.y_min) / temp.height
        temp['yolo'] = temp.apply(lambda x: '{} '.format(str(TARGETS[x[3]]) + ' ' + ' '.join([str(round(y, 4)) for y in x[-4:]])), axis=1)
        
        data3 = pd.concat([data3, temp], ignore_index=True)

In [357]:
data3['pose'] = 'in'

In [358]:
data_all = pd.concat([
    data[~((data.path.isin(data3.path.unique()))&(data.label == 'caries'))], 
    data3], ignore_index=True)

In [359]:
data_all.label.value_counts()

teeth     5585
caries     670
Name: label, dtype: int64

In [360]:
data.label.value_counts()

teeth     5585
caries     609
Name: label, dtype: int64

In [361]:
data_all.pose.value_counts()

out    350
in     320
Name: pose, dtype: int64

In [367]:
data_all = data_all[~data_all.path.isin(data_all[data_all.pose == 'out'].path.unique())]

## Формат йоло

In [368]:
data[['file', 'yolo', 'label']].rename(columns={'file': 'id', 'label': 'class'}).head()

,id,yolo,class
0,823.jpg,0 0.8789 0.6699 0.1875 0.207,teeth
1,823.jpg,0 0.9238 0.8848 0.1523 0.2227,teeth
2,189.jpg,0 0.5605 0.3594 0.0742 0.0469,teeth
3,189.jpg,0 0.4727 0.3594 0.0859 0.0469,teeth
4,189.jpg,0 0.4102 0.3652 0.0391 0.0352,teeth


In [371]:
data_all = data_all[['file', 'yolo', 'label']].rename(columns={'file': 'id', 'label': 'class'})

In [372]:
train_all, val_all = train_test_split(data_all, test_size=0.15, stratify=data_all['class'],
                                      random_state=17)

In [375]:
for image_file in tqdm(train_all.id.unique()):
    
    temp_ann = train_all[train_all.id == image_file]
    image = cv2.imread('data/images/'+temp_ann.id.iloc[0])
    
    s_ann = ''
    
    for i, val in temp_ann.iterrows():
        
        s_ann += val['yolo']
        if i != temp_ann.shape[0] - 1:
            s_ann += '\n'
    
    label_file = 'yolo_data/train/labels/'+image_file[:-4]+'.txt'
    image_file = 'yolo_data/train/images/'+image_file
    
    with open(label_file, 'w') as f:
        f.write(s_ann)
    
    cv2.imwrite(image_file, image)

100%|████████████████████████████████████████| 613/613 [00:01<00:00, 390.43it/s]


In [376]:
for image_file in tqdm(val_all.id.unique()):
    
    temp_ann = val_all[val_all.id == image_file]
    image = cv2.imread('data/images/'+temp_ann.id.iloc[0])
    
    s_ann = ''
    
    for i, val in temp_ann.iterrows():
        
        s_ann += val['yolo']
        if i != temp_ann.shape[0] - 1:
            s_ann += '\n'
    
    label_file = 'yolo_data/val/labels/'+image_file[:-4]+'.txt'
    image_file = 'yolo_data/val/images/'+image_file
    
    with open(label_file, 'w') as f:
        f.write(s_ann)
    
    cv2.imwrite(image_file, image)

100%|████████████████████████████████████████| 341/341 [00:00<00:00, 478.20it/s]


# Обучение YOLO

In [377]:
import os

In [378]:
yaml_file = "yolov5/data/ekb.yaml"
train_images_dir = os.path.join('..', 'yolo_data/train', 'images')
val_images_dir = os.path.join('..', 'yolo_data/val', 'images')

names_str = ''
for item in list(TARGETS.keys()):
    names_str = names_str + ", \'%s\'"%item
names_str = "names: ["+names_str[1:]+"]"

with open(yaml_file, "w") as wobj:
    wobj.write("train: %s\n"%train_images_dir)
    wobj.write("val: %s\n"%val_images_dir)
    wobj.write("nc: %d\n"%len(list(TARGETS.values())))
    wobj.write(names_str+"\n")
    

In [ ]:
!cd yolov5 && python train.py --img 256 --batch 8 --epochs 50 --data ../yolov5/data/ekb.yaml --weights yolov5m.pt
# --weights ../yolov5/runs/train/exp3/weights/last.pt



train: weights=yolov5m.pt, cfg=, data=../yolov5/data/ekb.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=8, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-261-g19f33cb Python-3.9.10 torch-1.11.0 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1,

In [288]:
!cd yolov5 && python detect.py --weights runs/train/exp7/weights/best.pt --img 256 --source ../data/images --augment --save-txt --conf-thres 0.5

detect: weights=['runs/train/exp7/weights/best.pt'], source=../data/images, data=data/coco128.yaml, imgsz=[256, 256], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=True, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-261-g19f33cb Python-3.9.10 torch-1.11.0 CPU

Fusing layers... 
Model summary: 290 layers, 20856975 parameters, 0 gradients, 48.0 GFLOPs
image 1/901 /Users/hare/Notebooks/Championships/CP/EKB/data/images/0.jpg: 256x256 2 teeths, Done. (0.162s)
image 2/901 /Users/hare/Notebooks/Championships/CP/EKB/data/images/0_ (1).jpg: 256x256 13 teeths, Done. (0.310s)
image 3/901 /Users/hare/Notebooks/Championships/CP/EKB/data/images/0_ (10).jpg: 256x256 16 teeths, Done. (0.237s)
image 4/901 /Users/hare/Notebooks/Championships/CP/EKB/data/im

In [ ]:
# for tta
# --augment